# Skill estimation using Stan

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(66)
import pickle

## Model that defined skill's distribution with more features

In [2]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  real scores[E];              // scores for each game
}
parameters {
  vector [N] skill;           // skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
    scores[i] ~ gamma( fabs(skill[PA[i]] - skill[PB[i]]) + 2, 0.5 );
  }   // win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [3]:
sm = pystan.StanModel(model_code = skill_model)

In [4]:
with open('skill_model_scores_diff.pkl', 'wb') as f: pickle.dump(sm, f)

In [5]:
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model_scores.pkl', 'rb'))
# except:  # ow, compile and save compiled model
#     sm = pystan.StanModel(model_code = skill_model)
#     with open('skill_model_scores.pkl', 'wb') as f: pickle.dump(sm, f)

## Processing data

In [48]:
def load_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins = []
    playerA, playerB = [], []
    raceA, raceB = [], []
    nplayers = len(playerid)
    nplays = np.zeros( (nplayers,nplayers) )
    race = {'P':0, 'T':1, 'Z':2}
    scoresA = []
    scoresB = []
    scores = []

    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                
                playerA.append(a+1)
                playerB.append(b+1)
                wins.append(1 if aw else 0) 
                raceA.append(race[csv[6]])
                raceB.append(race[csv[7]])
                sa, sb = csv[3].split('–')
                scores.append(abs(int(sa)-int(sb)))
#                 scores.append((int(sa)-int(sb)))
                scoresA.append(int(sa))
                scoresB.append(int(sb))

    return nplayers,playerA,playerB,raceA, raceB, wins, scores

In [49]:
nplayers,playerA,playerB,raceA, raceB, wins, scores = load_data()

In [50]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])
print('scores', scores[:10])

summary: 
# players 999
# games 4678
player A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
player B [2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
wins [0, 0, 0, 0, 1, 1, 1, 0, 0, 1]
scores [2, 1, 2, 1, 1, 1, 1, 2, 2, 1]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [51]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.8,
    'win':wins,
    'PA': playerA,
    'PB': playerB,
    'scores': scores
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [52]:
fit = sm.sampling(data=skill_data, iter=2000, chains=4, n_jobs = 4, verbose = True)

In [53]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [54]:
samples

OrderedDict([('skill',
              array([[ 0.03043601, -0.01945221,  0.2372419 , ...,  0.40546936,
                       0.26976411,  0.31870848],
                     [ 0.3407881 ,  0.29814437,  0.39220335, ...,  0.39217658,
                       0.16117625,  0.50410268],
                     [ 0.18968069,  0.13598773,  0.35067451, ..., -0.43486911,
                       0.11033042, -0.22992849],
                     ...,
                     [ 0.09752881,  0.20026908,  0.26025287, ..., -0.17821503,
                       0.57858063, -0.11471914],
                     [ 0.02077048,  0.12079096,  0.24635477, ...,  0.60021298,
                      -0.0200149 , -0.05199799],
                     [ 0.1817821 ,  0.23870741,  0.04972495, ..., -1.02211973,
                       0.14432188,  0.19489678]])),
             ('lp__',
              array([-8571.97956435, -8525.23347056, -8584.93998095, ...,
                     -8546.39307873, -8535.92322155, -8496.53554505]))])

In [55]:
samples['skill'].shape # 2*100 iterations? 999 players

(4000, 999)

In [56]:
samples['skill'].mean(0)

array([ 1.68346001e-01,  1.71184490e-01,  2.10017834e-01,  2.00063215e-01,
        1.56450245e-01,  1.50044754e-01,  1.33874968e-01,  1.51094879e-01,
        1.77418532e-01,  2.07133931e-01,  8.43078394e-02,  1.74607697e-01,
        1.55924296e-01,  1.03476090e-01,  1.84425048e-01,  1.19334350e-01,
       -2.12448813e-02, -1.40260593e-02,  4.32114215e-02,  1.94613797e-01,
        2.39988277e-02,  1.46242257e-01,  1.28563712e-01,  1.26262359e-01,
        1.88919194e-01,  1.70651404e-01,  1.60271985e-01,  8.18558233e-02,
        1.53455562e-01,  1.36987411e-01,  1.86090836e-01, -7.04003140e-02,
        1.01773091e-01,  2.13766055e-01,  9.71835733e-02,  2.76266332e-01,
        6.57114880e-02,  1.39589609e-01,  1.74740391e-01,  2.06648954e-01,
        5.81915986e-02,  1.69382480e-01,  1.33273134e-01,  1.73061255e-01,
        6.75812771e-02,  1.94877585e-01,  1.87740100e-01,  1.68392151e-01,
        1.98158006e-01,  1.78104162e-01,  8.36431054e-02,  1.14351672e-01,
        1.74881507e-01,  

Remember to save the prediction!

In [57]:
with open('skill_hat_scores_diff.pkl', 'wb') as f: 
    pickle.dump(samples, f)

## Sample Model Evaluation

In [58]:
skill_hat = pickle.load(open('skill_hat_scores_diff.pkl', 'rb'))

In [59]:
skill_hat = skill_hat['skill']

In [60]:
def load_valid_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    nplayers = len(playerid)
    playername = ['']*nplayers
    for player in playerid:
        playername[ playerid[player] ]=player  # id to name lookup


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    race = {'P':0, 'T':1, 'Z':2}
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                nwins[a,b] += aw
                nwins[b,a] += bw
                games.append((a,b,aw,race[csv[6]],race[csv[7]]))
    
    return nplayers, nplays, nwins, games

In [61]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data()

In [62]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 9542.0
(999, 999) 4788.0
games 4771


In [63]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [64]:
loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [65]:
loss

(0.4533695803335238, 0.40181268882175225)